In [1]:
import pandas as pd
import xlsxwriter
from os import walk
from def_zhanshi import *
from def_fiveTables import *
from def_fiveTables2 import *
from def_SummarySheets import *
from combin2csv import *
from datetime import datetime,timedelta
import time
# 忽略无用警告
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

d:\python\envs\jupyterpy36\scripts\python.exe


In [2]:
print("before set display.max_rows = ",pd.get_option("display.max_rows"))
pd.set_option("display.max_rows",80)
print("after set display.max_rows = ",pd.get_option("display.max_rows"))

before set display.max_rows =  8000
after set display.max_rows =  80


In [3]:
pd.set_option('display.large_repr', 'info')

In [4]:
import numpy as np
np.set_printoptions(suppress=True, threshold=np.nan)

In [5]:
disk = "G"
date = '19/06/22'  # 表中日期
date1 = '190622'  # 文件名日期
date2 = '2019-06-22'
date3 = '190621'
filename = 'testALL'+ date1+'.xlsx'  # 输出文件名
inputPath = disk +":/work/daily/DataShow/190622/"  # 输出路径
inputPath2 = disk + ":/work/daily/DataShow/"

In [6]:
san_days  = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=3)).strftime('%Y-%m-%d')
qi_days   = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')
erwu_days = (datetime.strptime(date2, '%Y-%m-%d') - timedelta(days=25)).strftime('%Y-%m-%d')
print("三日前：" + str(san_days))
print("七日前：" + str(qi_days))
print("二十五日前：" + str(erwu_days))

三日前：2019-06-19
七日前：2019-06-15
二十五日前：2019-05-28


In [7]:
df1 = pd.read_csv(inputPath + "data1.csv")
# df1["销售品编号"] = round(df1["销售品编号"],0)
# df1 = pd.read_excel(inputPath + "data1.xlsx")
split1 = pd.DataFrame((x.split(' ') for x in df1['订单生成时间']),index=df1.index,columns=['订单生成日期','订单生成小时'])
df1 = pd.merge(df1, split1, left_index=True, right_index=True)
# df1.to_csv(inputPath+"df1.csv",index=False)

In [8]:
df1["销售品编号"][0:10]

0    10011228962
1    10011212654
2    10011228970
3    10011228970
4    10011228990
5    10011228970
6    10011228964
7    10011212654
8    10011228970
9    10011212567
Name: 销售品编号, dtype: int64

In [9]:
df2 = pd.read_csv(inputPath + "NewPro.csv")
# df2 = pd.read_excel(inputPath + "NewPro.xlsx")
split2 = pd.DataFrame((x.split(' ') for x in df2['订单生成时间']),index=df2.index,columns=['订单生成日期','订单生成小时'])
df2 = pd.merge(df2, split2, left_index=True, right_index=True)


In [10]:
df3 = pd.read_csv(inputPath2 + "CHZH/CHZH_Info"+ date3 + ".csv") # 首充历史表


In [11]:
df4 = pd.read_csv(inputPath + "NewPay" + date1 + ".csv") # 当日历史表
split4 = pd.DataFrame((x.split(' ') for x in df4['订单生成时间']), index=df4.index, columns=['订单生成日期', '订单生成小时'])
df4 = pd.merge(df4, split4, left_index=True, right_index=True)

In [12]:
df5 = pd.read_csv(inputPath + "NewJD.csv")
split5 = pd.DataFrame((x.split(' ') for x in df5['用户下单时间']),index=df5.index,columns=['用户下单日期','用户下单小时'])
df5 = pd.merge(df5, split5, left_index=True, right_index=True)


In [13]:
df1 = df1[df1["订单生成日期"] == san_days]
df2 = df2[df2["订单生成日期"] == san_days]
df5 = df5[df5["用户下单日期"] == san_days] # 京东表

In [14]:
print("df1:"+str(df1.shape))
print("df2:"+str(df2.shape))
print("df3:"+str(df3.shape))
print("df4:"+str(df4.shape))
print("df5:"+str(df5.shape))

df1:(0, 97)
df2:(0, 37)
df3:(72310, 97)
df4:(32775, 97)
df5:(15290, 38)


In [ ]:
df1 = df1[(True ^ df1['订单状态'].isin(['办理成功', '办理中', '办理失败']))]
df1 = df1[pd.notnull(df1['所在省 / 市 / 县'])]
df1 = df1[df1["订单编号"].str.contains('订单编号') == False]
df1 = df1.drop_duplicates(subset=['订单编号'], keep='first')
df1 = df1[pd.notnull(df1['号码归属地'])]
df1 = df1[pd.notnull(df1['入网手机号'])]
df1["入网手机号"] = df1['入网手机号'].astype('int64')

In [ ]:
df2 = df2[df2["模式分类"] == "京东模式"]
df2 = df2[pd.notnull(df2['是否下省'])]
df2.shape

In [ ]:
df34 = pd.concat([df3, df4],sort=False)
df34 = df34.drop_duplicates(subset=['主号码'], keep='first') # 主号码
df34 = df34[pd.notnull(df34['主号码'])]
df34["主号码"] = df34['主号码'].astype('int64')
print("当前日期：" + date2)
ggDays = get_nday_list(date2, 7)
df34 = df34[df34["订单生成日期"].isin(ggDays)]


In [ ]:
df51 = df5[["运营商单号", "物流单号"]]
df52 = df5[["运营商单号", "APP操作时间"]]
df51 = df51[pd.notnull(df51['物流单号'])]
df52 = df52[pd.notnull(df52['APP操作时间'])]

In [ ]:
print("df51:"+str(df51.shape))
print("df52:"+str(df52.shape))

In [ ]:
df11 = pd.merge(df1, df2[["订单编号", "模式分类", "是否下省"]],
                left_on="订单编号", right_on="订单编号", how="left")
df11 = pd.merge(df11, df51, left_on="订单编号", right_on="运营商单号", how="left")
df11 = pd.merge(df11, df52, left_on="订单编号", right_on="运营商单号", how="left")
df11['入网手机号'] = df11['入网手机号'].apply(int)
df11 = pd.merge(df11, df34[['主号码']],left_on="入网手机号", right_on="主号码", how="left") #'订单编号',
# df11 = df11[df11["订单状态"] == "交易完成"]
df11["主号码_y"] = df11["主号码_y"].fillna(-1)
df11["主号码_y"] = df11['主号码_y'].astype('int64')

In [ ]:
df12 = df11.copy()

In [ ]:
df12.shape

In [ ]:
def shouchong(x):
    if x == -1:
        return 0
    else:
        return 1

In [ ]:
def laidan(x):
    if x is np.nan:
        return 0
    else:
        return 1


def fahuo(a,b,c):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    elif a is np.nan and c is not np.nan:
        return 1
    else:
        return 0

def qianshou(a,b):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    else:
        return 0

def jihuo(x):
    if x == "交易完成":
        return 1
    else:
        return 0

def paidan(x):
    if x is np.nan:
        return 0
    else:
        return 1

In [ ]:
df11['派单'] = df11["是否下省"].apply(paidan)
df11['派卡'] = df11["物流单号_y"].apply(paidan)
df11['上门'] = df11["APP操作时间"].apply(paidan)
df11['首充'] = df11["主号码_y"].apply(shouchong) #订单编号_y
df11['来单量'] = df11['订单编号'].apply(laidan) #订单编号_x
df11['发货量'] = df11.apply(lambda x: fahuo(x["物流单号_x"], x["订单状态"], x["物流签收时间"]), axis=1)
df11['签收量'] = df11.apply(lambda x: qianshou(x["物流签收时间"], x["订单状态"]), axis=1)
df11['激活量'] = df11["订单状态"].apply(jihuo)

In [ ]:
df = df11[["号码归属地","销售品编号","营业厅送货方式","派单","派卡","上门","首充","发货量","签收量","激活量","来单量"]]
split1 = pd.DataFrame((x.split('/') for x in df['号码归属地']),index=df.index,columns=['所属省','所属市'])
df = pd.merge(df, split1, left_index=True, right_index=True)

In [ ]:
# df["销售品编号"] = df["销售品编号"].map(lambda x : str(x))

In [ ]:
df.head()

In [ ]:
# 产品标卡
biaoka = pd.read_excel(inputPath2 + "产品标卡.xlsx")
# biaoka["销售品编号"] = biaoka["销售品编号"].map(lambda x: str(x))

In [ ]:
biaoka["销售品编号"][0:5]

In [ ]:
# df["销售品编号"] = df["销售品编号"].map(lambda x : x[:-2])

In [ ]:
df["销售品编号"][0:5]

In [ ]:
data_JM3 = pd.merge(df, biaoka, how="left", on="销售品编号")


In [ ]:
# data_JM3.to_excel(inputPath+"data_JM31.xlsx",index=False)

In [ ]:
data_JM3.columns

In [ ]:
data10 = JD_mode_product(data_JM3)


In [ ]:
data10.info()


In [ ]:
data10.to_excel(inputPath+"data10_1.xlsx",index=False)

In [ ]:
223*223+355*355